In [ ]:
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt

In [ ]:
# spiral data with noise
np.random.seed(42)
N=200
noisefact=0.7
p = sorted(np.random.random(N)*np.pi*5)
v1 = p*np.cos(p)+np.random.randn(N)*noisefact
v2 = p*np.sin(p)+np.random.randn(N)*noisefact
p2 = sorted(np.random.random(N)*np.pi*5+np.pi)
w1 = p*np.cos(p2)+np.random.randn(N)*noisefact
w2 = p*np.sin(p2)+np.random.randn(N)*noisefact
# visualize
plt.scatter(v1, v2)
plt.scatter(w1, w2)
y1 = np.zeros((N, 1))
y2 = np.ones((N, 1))

In [ ]:
# pack into feature arrays
#
X = np.stack([np.concatenate([v1, w1]), np.concatenate([v2, w2])], axis = 1)
y = np.concatenate([y1, y2]).ravel()
# normalize
X -= X.mean(axis=0)
X /= X.std(axis=0)

In [ ]:
def visualize_classifier(predict, xmin, xmax, ymin, ymax, **kwargs):
    xx, yy = np.meshgrid(
        np.linspace(xmin, xmax, 100),
        np.linspace(ymin, ymax, 100),
    )
    X = np.stack([xx, yy], axis=-1).reshape(-1, 2)
    zz = predict(X).reshape(xx.shape)
    plt.contourf(xx, yy, zz, levels=100, **kwargs)

In [ ]:
# convert to torch tensor
X, y = torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [ ]:
# build torch model

neurons = 32
model = nn.Sequential(
    nn.Linear(2, neurons), nn.ReLU(), 
    nn.Linear(neurons, 1), nn.Sigmoid()
)


In [ ]:
#optimizer = torch.optim.SGD(model.parameters(), lr=1.0)
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
history = []
for i in range(100):
    optimizer.zero_grad()
    y_pred = model(X).squeeze(1)
    loss = nn.functional.binary_cross_entropy(y_pred, y)
    loss.backward()
    history.append(loss.item())
    optimizer.step()

In [ ]:
plt.plot(history)

In [ ]:
with torch.no_grad():
    visualize_classifier(
        lambda x: model(torch.tensor(x, dtype=torch.float32)).squeeze(1).numpy(),
        -3,3,-3,3,
        cmap="RdBu"
    )
plt.scatter(*X[y==0].T, color="red")
plt.scatter(*X[y==1].T, color="blue")

**Accuracy:**

In [ ]:
with torch.no_grad():
    acc = ((model(X).squeeze().numpy() > 0.5) == y.numpy()).mean()
    print(acc)